A playaround to test out the persistent vectorstore functionality of Qdrant before the further development of RAG based tool  

In [4]:
pip install qdrant-client[fastembed]

     ---------------------------------------- 0.0/254.1 kB ? eta -:--:--
     -------------------------------------- 254.1/254.1 kB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------- ----------- 0.8/1.1 MB 16.1 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 13.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/62.5 kB ? eta -:--:--
     ---------------------------------------- 62.5/62.5 kB 3.3 MB/s eta 0:00:00
  Using cached tokenizers-0.15.2-cp310-none-win_amd64.whl (2.2 MB)
  Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
     ---------------------------------------- 0.0/426.9 kB ? eta -:--:--
     ------------------------------------- 426.9/426.9 kB 13.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
  Attempting uninstall: protobuf
    

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.42.3 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.42.3 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.
proto-plus 1.23.0 requires protobuf<5.0.0dev,>=3.19.0, but you have protobuf 5.27.2 which is incompatible.
opentelemetry-proto 1.25.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.2 which is incompatible.
grpc-google-iam-v1 0.13.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-vision 3.7.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-resour

In [60]:
from qdrant_client import QdrantClient
from dotenv import dotenv_values

config = dotenv_values(".env")


# Initialize the client
# client = QdrantClient(":memory:") # For small experiments
qdrant_client = QdrantClient(
    url=config["QDRANT_ENDPOINT"], 
    api_key=config["QDRANT_API_KEY"],
)

# Prepare your documents, metadata, and IDs
docs = ["Qdrant has Langchain integrations", "Qdrant also has Llama Index integrations"]
metadata = [
    {"source": "Langchain-docs"},
    {"source": "Llama-index-docs"},
]
ids = [42, 2]

# If you want to change the model:
# client.set_model("sentence-transformers/all-MiniLM-L6-v2")
# List of supported models: https://qdrant.github.io/fastembed/examples/Supported_Models

# Use the new add() instead of upsert()
# This internally calls embed() of the configured embedding model
qdrant_client.add(
    collection_name="collection1_add",
    documents=docs,
    metadata=metadata,
    ids=ids
)

search_result = qdrant_client.query(
    collection_name="demo_collection",
    query_text="This is a query document"
)
print(search_result)

[QueryResponse(id=42, embedding=None, sparse_embedding=None, metadata={'document': 'Qdrant has Langchain integrations', 'source': 'Langchain-docs'}, document='Qdrant has Langchain integrations', score=0.8133327), QueryResponse(id=2, embedding=None, sparse_embedding=None, metadata={'document': 'Qdrant also has Llama Index integrations', 'source': 'Llama-index-docs'}, document='Qdrant also has Llama Index integrations', score=0.81049025)]


In [61]:
from qdrant_client import QdrantClient, models

qdrant_client.create_collection(
    collection_name="collection2_create",
    vectors_config=models.VectorParams(
        size=1024,
        distance=models.Distance.DOT,
    ),
)

True

In [62]:
qdrant_client.get_collections().collections

[CollectionDescription(name='demo_collection'),
 CollectionDescription(name='test_collection'),
 CollectionDescription(name='collection2_create'),
 CollectionDescription(name='collection1_add')]

In [73]:
collection_info = qdrant_client.get_collection(collection_name="collection1_add")
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', None),
 ('indexed_vectors_count', 0),
 ('points_count', 2),
 ('segments_count', 2),
 ('config',
  CollectionConfig(params=CollectionParams(vectors={'fast-bge-small-en': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConf

In [74]:
collection_info = qdrant_client.get_collection(collection_name="collection2_create")
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', None),
 ('indexed_vectors_count', 0),
 ('points_count', 10),
 ('segments_count', 2),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_

In [67]:
qdrant_client.delete_collection(collection_name='test_collection')

False

In [6]:
notes = [
    {
        "title": "Project Alpha Review",
        "text": "Review the current progress of Project Alpha, focusing on the integration of the new API. Check for any compatibility issues with the existing system and document the steps needed to resolve them. Schedule a meeting with the development team to discuss the timeline and any potential roadblocks."
    },
    {
        "title": "Learning Path Update",
        "text": "Update the learning path document with the latest courses on React and Node.js from Pluralsight. Schedule at least 2 hours weekly to dedicate to these courses. Aim to complete the React course by the end of the month and the Node.js course by mid-next month."
    },
    {
        "title": "Weekly Team Meeting Agenda",
        "text": "Prepare the agenda for the weekly team meeting. Include the following topics: project updates, review of the sprint backlog, discussion on the new feature requests, and a brainstorming session for improving remote work practices. Send out the agenda and the Zoom link by Thursday afternoon."
    },
    {
        "title": "Code Review Process Improvement",
        "text": "Analyze the current code review process to identify inefficiencies. Consider adopting a new tool that integrates with our version control system. Explore options such as GitHub Actions for automating parts of the process. Draft a proposal with recommendations and share it with the team for feedback."
    },
    {
        "title": "Cloud Migration Strategy",
        "text": "Draft a plan for migrating our current on-premise infrastructure to the cloud. The plan should cover the selection of a cloud provider, cost analysis, and a phased migration approach. Identify critical applications for the first phase and any potential risks or challenges. Schedule a meeting with the IT department to discuss the plan."
    },
    {
        "title": "Quarterly Goals Review",
        "text": "Review the progress towards the quarterly goals. Update the documentation to reflect any completed objectives and outline steps for any remaining goals. Schedule individual meetings with team members to discuss their contributions and any support they might need to achieve their targets."
    },
    {
        "title": "Personal Development Plan",
        "text": "Reflect on the past quarter's achievements and areas for improvement. Update the personal development plan to include new technical skills to learn, certifications to pursue, and networking events to attend. Set realistic timelines and check-in points to monitor progress."
    },
    {
        "title": "End-of-Year Performance Reviews",
        "text": "Start preparing for the end-of-year performance reviews. Collect feedback from peers and managers, review project contributions, and document achievements. Consider areas for improvement and set goals for the next year. Schedule preliminary discussions with each team member to gather their self-assessments."
    },
    {
        "title": "Technology Stack Evaluation",
        "text": "Conduct an evaluation of our current technology stack to identify any outdated technologies or tools that could be replaced for better performance and productivity. Research emerging technologies that might benefit our projects. Prepare a report with findings and recommendations to present to the management team."
    },
    {
        "title": "Team Building Event Planning",
        "text": "Plan a team-building event for the next quarter. Consider activities that can be done remotely, such as virtual escape rooms or online game nights. Survey the team for their preferences and availability. Draft a budget proposal for the event and submit it for approval."
    }
]

In [69]:
import cohere
import uuid
from dotenv import dotenv_values

cohere_api_key = dotenv_values(".env")["COHERE_API_KEY"]
cohere_client = cohere.Client(api_key=cohere_api_key)

response = cohere_client.embed(
    texts=[
        note.get("text")
        for note in notes
    ],
    model="embed-multilingual-v3.0",
    input_type="search_document",
)

qdrant_client.upsert(
    collection_name="collection2_create",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector=embedding,
            payload=note,
        )
        for note, embedding in zip(notes, response.embeddings)
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [75]:
for note, embedding in zip(notes, response.embeddings):
    point_id = uuid.uuid4().hex  # Generate and store the UUID
    print(point_id)  # Print the generated UUID
    qdrant_client.upsert(
        collection_name="collection2_create",
        points=[
            models.PointStruct(
                id=point_id,
                vector=embedding,
                payload=note,
            )
        ]
    )

6f0366935c52430bbf80be49c47a9e72
8591718bdaf6462fbb6b7f58b0401d88
17b57d21405343be8f9f2168bc13bc35
39c9dc4930b14bde92741e455823bfb3
c804cf7e1fde4a4199efe8217f66d7a7
cb2d592527f44ce9b0800ce8e9225b27
1749dc58a16246fe8d4f023319ebe905
a42435330b0640d38c150a9c78b15946
9298782345624cf8a165ed8fabc5cbe2
bbfe2b58898941248614b2e67bb9def2


In [80]:
qdrant_client.retrieve(
    collection_name="collection2_create",
    ids=["6f0366935c52430bbf80be49c47a9e72"],
    with_vectors=True # the default is False
)

[Record(id='6f036693-5c52-430b-bf80-be49c47a9e72', payload={'text': 'Review the current progress of Project Alpha, focusing on the integration of the new API. Check for any compatibility issues with the existing system and document the steps needed to resolve them. Schedule a meeting with the development team to discuss the timeline and any potential roadblocks.', 'title': 'Project Alpha Review'}, vector=[0.018081665, 0.03137207, -0.035491943, 0.03555298, -0.017929077, -0.034088135, -0.0032920837, -0.026763916, -0.0054626465, -0.011558533, -0.008255005, 0.021072388, 0.00447464, 0.0051231384, 0.010604858, 0.015380859, 0.022994995, -0.0362854, -0.0062217712, -0.023468018, -0.0289917, 0.04019165, 0.07330322, -0.049835205, 0.02168274, 0.0010652542, 0.033721924, 0.0030727386, 0.0552063, 0.0013885498, 0.001335144, -0.03074646, 0.03111267, 0.017791748, 0.038909912, 0.07110596, -0.016937256, -0.015975952, 0.0064086914, 0.06311035, -0.04550171, 0.00040650368, -0.021255493, 0.06439209, -0.008445